In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow

from tensorflow import keras
from tensorflow.keras import callbacks
from uproot_io import Events, View
from tensorflow.keras.models import load_model

In [2]:
# Creating Event objects for the data
events1 = Events("/Users/mohammed/code/neutrinos-dune-files/CheatedRecoFile_1.root")
events2 = Events("/Users/mohammed/code/neutrinos-dune-files/CheatedRecoFile_2.root")
events3 = Events("/Users/mohammed/code/neutrinos-dune-files/CheatedRecoFile_3.root")
events4 = Events("/Users/mohammed/code/neutrinos-dune-files/CheatedRecoFile_4.root")

In [3]:
# Getting the event indices and finding the number of events in each file
numbers1 = np.array(list(set(events1.event_number)))
numbers2 = np.array(list(set(events2.event_number)))
numbers3 = np.array(list(set(events3.event_number)))
numbers4 = np.array(list(set(events4.event_number)))
len(numbers1), len(numbers2), len(numbers3), len(numbers4)

(9310, 9265, 9310, 9257)

In [4]:
# Creating view classes for each event
view_w1 = View(events1, "w")
view_w2 = View(events2, "w")
view_w3 = View(events3, "w")
view_w4 = View(events4, "w")

In [5]:
# Functions to extract the truth record for each event

def labelling(numbers, events):
    '''Labels muon neutrino events as: 0
       Labels electron neutrino events as: 1
       Labels neutral current events as: 2'''
    label = []
    
    for n in numbers:
        mcp = []
        index = np.where(events.event_number == n)[0]

        for i in index:
            parent = events.reco_parent_index[i]
            if parent == 0:
                mcp.append(events.mc_pdg[i])

        if 13 in mcp or -13 in mcp:
            label.append(0)

        elif 11 in mcp or -11 in mcp:
            label.append(1)    
            
        else:
            label.append(2)
            
    return np.array(label)

def process_numbers(numbers, events):
    '''Labels muon neutrino events as: [1,0,0]
       Labels electron neutrino events as: [0,1,0]
       Labels neutral current events as: [0,0,1]'''
    labels = []
    for n in numbers:
        mcp = []
        index = np.where(events.event_number == n)[0]

        for i in index:
            parent = events.reco_parent_index[i]
            if parent == 0:
                mcp.append(events.mc_pdg[i])

        if 11 in mcp or -11 in mcp:
            labels.append([0, 1, 0])
        elif 13 in mcp or -13 in mcp:
            labels.append([1, 0, 0])
        else:
            labels.append([0, 0, 1])
    return labels

def oneHotLabelling(numbers_list, events_list):
    labels = []
    for numbers, events in zip(numbers_list, events_list):
        labels.extend(process_numbers(numbers, events))
    return np.array(labels).astype(float)

In [13]:
labels1 = oneHotLabelling([numbers1], [events1])

In [18]:
# Making the event images suitable for a CNN

def image(numbers_list, views_list):
    """Creates 128 by 128 pixels images for each event, making sure the interaction vertex is included."""
    images = []
    
    for numbers, view_w in zip(numbers_list, views_list):
        for n in numbers:
            x, z, adc, vx, vz = view_w.x[n], view_w.z[n], view_w.adc[n], view_w.true_vtx_x[n], view_w.true_vtx_z[n]

            matrix, _, _ = np.histogram2d(x, z, bins=256, weights=adc)
            matrix = (matrix > 0).astype(np.uint8).reshape(256, 256, 1)
            images.append(matrix)

    return np.stack(images)

In [19]:
images1 = image([numbers1], [view_w1])

In [10]:
cnn_model = load_model('modelv1')

2024-02-16 09:22:34.869293: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-16 09:22:34.869596: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-16 09:22:34.869924: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-16 09:22:34.870774: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-16 09:22:34.871970: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
loss, accuracy = cnn_model.evaluate(images1, labels1)

203/291 [===================>..........] - ETA: 2s - loss: 2.7950 - accuracy: 0.3941